In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 24.7MB/s 
     |████████████████████████████████| 901kB 45.0MB/s 


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report


model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


# URLs for dataset- 

Amazon data : https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/amazon_cells_labelled.txt

Yelp data : https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/yelp_data.txt

# Type the name of the dataset in 'url', 
# like this -> url = amazon_data

In [ ]:
amazon_data = "https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/amazon_cells_labelled.txt" 

yelp_data = "https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/yelp_data.txt"

url = yelp_data

In [ ]:
df = pd.read_csv(url, header=None, delimiter="\t")

sen = df[0]
label = df[1]

In [ ]:
train_sentences = sen[:950]
train_labels = label[:950]

valid_sentences = sen[950:995]
valid_labels = label[950:995]

test_sentences = sen[995:]
test_labels = label[995:]

In [ ]:
train_sen, train_lab = [],[]
for i in train_sentences:
  train_sen.append(i)
for i in train_labels:
  train_lab.append(i)

train_sen = np.array(train_sen)
train_lab = np.array(train_lab)

valid_sen, valid_lab = [],[]
for i in valid_sentences:
  valid_sen.append(i)
for i in valid_labels:
  valid_lab.append(i)

train_sen = np.array(train_sen)
train_lab = np.array(train_lab)

valid_sen = np.array(valid_sen)
valid_lab = np.array(valid_lab)

train_input = pd.DataFrame([train_sen,train_lab]).T
valid_input = pd.DataFrame([valid_sen,valid_lab]).T


train_input.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
valid_input.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
# train_input['DATA_COLUMN'] = train_input['DATA_COLUMN'].str.decode("utf-8")
print(train_input.head())
print(valid_input.head())

                                         DATA_COLUMN LABEL_COLUMN
0                           Wow... Loved this place.            1
1                                 Crust is not good.            0
2          Not tasty and the texture was just nasty.            0
3  Stopped by during the late May bank holiday of...            1
4  The selection on the menu was great and so wer...            1
                                         DATA_COLUMN LABEL_COLUMN
0         It wasn't busy at all and now we know why.            0
1  The tables outside are also dirty a lot of the...            0
2  The ambiance here did not feel like a buffet s...            0
3                               Con: spotty service.            0
4  The fries were not hot, and neither was my bur...            0


In [ ]:
def convert_data_to_examples(train, valid, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = valid.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           valid, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_input, valid_input, DATA_COLUMN, LABEL_COLUMN)
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data, verbose=2)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

# Give a name to the model

In [2]:
model_name = "TYPEHERE" # Don't give space between the words

model.save('/content/'+model_name)

NameError: ignored

In [3]:
def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths 

# Save the model and download the model to local drive

In [4]:
directory = "/content/"+model_name+"/"

file_paths = get_all_file_paths(directory)

with ZipFile('/content/'+model_name+'.zip','w') as zip:
        for file_ in file_paths:
            zip.write(file_)

files.download('/content/'+model_name+'.zip')

NameError: ignored

# Predict Results

In [1]:
pred_sentences = ['Nice phone']
pred_labels = [1]

test_sentences.extend(pred_sentences)
test_labels.extend(pred_label)

tf_batch = tokenizer(test_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = [0,1]

label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(test_sentences)):
  print(test_sentences[i], ": ", labels[label[i]])

NameError: ignored

# Scikit-learn Classification Report

In [6]:
print(classification_report(test_labels, label))

NameError: ignored